In [0]:
!pip install gensim

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 38.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import os
import shutil
import sys
import zipfile
import gensim
import re
import warnings
warnings.filterwarnings('ignore')

# Data Loading
## Data from the Kaggle Synthetic dataset from Inscribo on Cyber-Security Attacks


In [0]:
# dims redux is the databricks workspace/catalog, csv_data is the volume
inscribo_cybersecurity_df = pd.read_csv('/Volumes/dims_redux/default/csv_data/cybersecurity_attacks.csv')

In [0]:
inscribo_cybersecurity_df.shape

(40000, 25)

In [0]:
# as the dataset is huge, we will take a sample of 1000 rows
inscribo_cybersecurity_df = inscribo_cybersecurity_df.sample(1000)

In [0]:
inscribo_cybersecurity_df.head()

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source
8281,2020-07-07 18:24:09,52.16.186.255,143.159.62.60,12191,18355,TCP,1480,Control,HTTP,Itaque necessitatibus iure dolores occaecati. ...,IoC Detected,33.27,NaN,Intrusion,Known Pattern B,Ignored,Medium,Hrishita Dave,Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3 like M...,Segment C,"Thane, Jharkhand",NaN,Log Data,NaN,Server
28232,2021-04-19 08:54:04,161.108.110.93,171.250.86.109,4193,62318,TCP,1032,Control,DNS,Iure facilis odit quisquam officia earum ratio...,NaN,92.29,Alert Triggered,Malware,Known Pattern B,Blocked,Low,Arhaan Amble,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment B,"Chennai, Manipur",89.236.139.44,Log Data,NaN,Server
14078,2021-12-05 14:27:16,2.28.162.6,29.219.167.39,32211,7377,ICMP,228,Control,HTTP,Modi porro excepturi totam cupiditate beatae f...,IoC Detected,13.57,Alert Triggered,DDoS,Known Pattern A,Logged,Low,Indranil Wable,Mozilla/5.0 (Windows 95) AppleWebKit/531.0 (KH...,Segment B,"Hapur, Assam",209.175.230.229,NaN,NaN,Firewall
16368,2020-08-08 18:36:47,54.22.11.21,37.134.249.227,11758,32857,UDP,1377,Data,HTTP,Deleniti eius commodi fugiat. Esse quae consec...,NaN,71.89,Alert Triggered,Intrusion,Known Pattern A,Ignored,High,Vritika Guha,Mozilla/5.0 (iPod; U; CPU iPhone OS 4_1 like M...,Segment A,"Fatehpur, Kerala",NaN,Log Data,NaN,Server
11039,2021-04-13 15:15:18,135.14.191.235,71.201.67.196,64579,49922,UDP,363,Data,DNS,Numquam accusamus maiores modi autem commodi. ...,NaN,52.72,NaN,Intrusion,Known Pattern A,Blocked,Low,Umang Krishnamurthy,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_1...,Segment B,"Hajipur, Tripura",25.82.156.123,NaN,NaN,Firewall


## Data from the National Gallery of Art's (NGA) OpenData Project


In [0]:
# importing latin american & non latin american seperately
# sampling the same amount as is in the latinamerican for the non-latinamerican (to make ratio 50:50),
# and then concatenating them 
latinamerican_art = pd.read_csv('/Volumes/dims_redux/default/csv_data/latinamerican_art.csv', on_bad_lines='skip')
nonlatinamerican_art = pd.read_csv('/Volumes/dims_redux/default/csv_data/non_latinamerican_art.csv', on_bad_lines='skip').sample(len(latinamerican_art))
nga_art_sample = pd.concat([latinamerican_art, nonlatinamerican_art])

In [0]:
nga_art_sample.head()

,iiifurl,iiifthumburl,accessioned,title,displayDate_created,roletype,role,forwarddisplayname,birthyear,deathyear,ulanid,artistofngaobject,nationality,constituenttype,beginyear_artistAssigned,endyear_artistAssigned,country_artistAssigned,zipcode_artistAssigned,medium,dimensions,inscription,markings,attribution,visualBrowserClassification,parentID,isVirtual,portfolio,series,volume,watermarks,uuid,viewtype,sequence,width,height,maxpixels,assistivetext,depictstmsobjectid,objectid,constituentid
0,https://api.nga.gov/iiif/00513eca-9795-4076-b5...,https://api.nga.gov/iiif/00513eca-9795-4076-b5...,1,Un poco alegre y graciosa (Somewhat gay and gr...,"Mexican, 1902 - 2002",artist,artist,Manuel Álvarez Bravo,1902,2002.0,500028115.0,1,Mexican,individual,NaN,NaN,NaN,NaN,gelatin silver print,image: 17.3 x 24.2 cm (6 13/16 x 9 1/2 in.)\nm...,"signed by artist, lower right in graphite: M. ...","lower left of mount, embossed Strathmore seal",Manuel Álvarez Bravo,photograph,153112.0,0.0,portfolio of 15 prints,NaN,NaN,NaN,00513eca-9795-4076-b5dd-63abfb5dd036,primary,0,4000,2871.0,640.0,NaN,150712,150712.0,16466.0
1,https://api.nga.gov/iiif/00513eca-9795-4076-b5...,https://api.nga.gov/iiif/00513eca-9795-4076-b5...,1,Un poco alegre y graciosa (Somewhat gay and gr...,"Mexican, 1902 - 2002",artist,artist,Manuel Álvarez Bravo,1902,2002.0,500028115.0,1,Mexican,individual,NaN,NaN,NaN,NaN,gelatin silver print,image: 17.3 x 24.2 cm (6 13/16 x 9 1/2 in.)\nm...,"signed by artist, lower right in graphite: M. ...","lower left of mount, embossed Strathmore seal",Manuel Álvarez Bravo,photograph,153112.0,0.0,portfolio of 15 prints,NaN,NaN,NaN,00513eca-9795-4076-b5dd-63abfb5dd036,primary,0,4000,2871.0,640.0,NaN,150712,150712.0,16466.0
2,https://api.nga.gov/iiif/00822cb7-0744-42c4-b6...,https://api.nga.gov/iiif/00822cb7-0744-42c4-b6...,1,Garden of Hecelchácan,"Mexican, 1908 - 2003",artist,artist,Alfredo Zalce,1908,2003.0,500065904.0,1,Mexican,individual,NaN,NaN,NaN,NaN,lithograph,NaN,NaN,NaN,Alfredo Zalce,print,NaN,0.0,Yucatan: no.8,Yucatan: no.8 [1946.21.182-189],NaN,NaN,00822cb7-0744-42c4-b68f-18219af95980,primary,0,4000,3405.0,640.0,NaN,33520,33520.0,3452.0
3,https://api.nga.gov/iiif/00822cb7-0744-42c4-b6...,https://api.nga.gov/iiif/00822cb7-0744-42c4-b6...,1,Garden of Hecelchácan,"Mexican, 1908 - 2003",artist,artist,Alfredo Zalce,1908,2003.0,500065904.0,1,Mexican,individual,NaN,NaN,NaN,NaN,lithograph,NaN,NaN,NaN,Alfredo Zalce,print,NaN,0.0,Yucatan: no.8,Yucatan: no.8 [1946.21.182-189],NaN,NaN,00822cb7-0744-42c4-b68f-18219af95980,primary,0,4000,3405.0,640.0,NaN,33520,33520.0,3452.0
4,https://api.nga.gov/iiif/015dcd20-3dda-448f-a5...,https://api.nga.gov/iiif/015dcd20-3dda-448f-a5...,1,Calavera Maderista,"Mexican, 1851 - 1913",artist,artist,José Guadalupe Posada,1851,1913.0,500032573.0,1,Mexican,individual,NaN,NaN,NaN,NaN,relief etching (zinc),NaN,NaN,NaN,José Guadalupe Posada,print,NaN,0.0,NaN,NaN,NaN,NaN,015dcd20-3dda-448f-a51b-618588eaff18,primary,0,1806,4000.0,NaN,NaN,30476,30476.0,5265.0


# Data Pre-Processing
## Processing data from Inscribo

In [0]:
inscribo_cybersecurity_df.columns

Index(['Timestamp', 'Source IP Address', 'Destination IP Address',
       'Source Port', 'Destination Port', 'Protocol', 'Packet Length',
       'Packet Type', 'Traffic Type', 'Payload Data', 'Malware Indicators',
       'Anomaly Scores', 'Alerts/Warnings', 'Attack Type', 'Attack Signature',
       'Action Taken', 'Severity Level', 'User Information',
       'Device Information', 'Network Segment', 'Geo-location Data',
       'Proxy Information', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source'],
      dtype='object')

In [0]:
inscribo_cybersecurity_df.describe() # describe helps identify the columns with numerical values as it calculates the mean, std, min, max, etc.

,Source Port,Destination Port,Packet Length,Anomaly Scores
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,33203.126000,33981.047000,805.197000,50.850130
std,18714.098998,18747.557063,408.876948,29.535216
min,1101.000000,1117.000000,64.000000,0.100000
25%,16371.500000,17864.000000,452.500000,25.375000
50%,32966.500000,32978.000000,799.500000,51.270000
75%,49695.250000,50554.000000,1163.250000,76.370000
max,65471.000000,65502.000000,1500.000000,99.930000


In [0]:
selected_cybersecurity_sample = inscribo_cybersecurity_df.loc[:, ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores', 'Source IP Address', 'Destination IP Address']]
selected_cybersecurity_sample.head()

,Source Port,Destination Port,Packet Length,Anomaly Scores,Source IP Address,Destination IP Address
8281,12191,18355,1480,33.27,52.16.186.255,143.159.62.60
28232,4193,62318,1032,92.29,161.108.110.93,171.250.86.109
14078,32211,7377,228,13.57,2.28.162.6,29.219.167.39
16368,11758,32857,1377,71.89,54.22.11.21,37.134.249.227
11039,64579,49922,363,52.72,135.14.191.235,71.201.67.196


### Filling in missing values


In [0]:
selected_cybersecurity_sample.isna().sum() # checking for null values, there is none

Source Port               0
Destination Port          0
Packet Length             0
Anomaly Scores            0
Source IP Address         0
Destination IP Address    0
dtype: int64

### Scaling the selected cybersecurity sample for use in Dimensionality Reduction
Scaling the Source IP and Destination IP address will not work because they are not linearly continous as are packet length and the anomaly scores. They are categorical in terms of data analysis because they are a string concatenation of multiple numerical sub-strings. In that case, we will only scale Source Port, Destination Port, Packet Length, and Anomaly Scores


In [0]:
from sklearn.preprocessing import StandardScaler
# saving the unscalable columns
selected_categorical_cybersecurity_sample = selected_cybersecurity_sample.loc[:, ['Source IP Address', 'Destination IP Address']]
# applying the standard scaler to the numerical data
scaler = StandardScaler()
selected_numerical_cybersecurity_sample = selected_cybersecurity_sample.loc[:, ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores']]
scaler.fit(selected_numerical_cybersecurity_sample)
scaled_numerical_cybersecurity_sample = scaler.transform(selected_numerical_cybersecurity_sample)
scaled_numerical_cybersecurity_sample = pd.DataFrame(scaler.transform(selected_numerical_cybersecurity_sample), columns=selected_numerical_cybersecurity_sample.columns)

In [0]:
scaled_numerical_cybersecurity_sample.head()

,Source Port,Destination Port,Packet Length,Anomaly Scores
0,-1.123358,-0.833915,1.651207,-0.595524
1,-1.550951,1.512257,0.554975,1.403768
2,-0.053041,-1.419777,-1.412371,-1.262858
3,-1.146508,-0.059987,1.399172,0.712722
4,1.677429,0.850720,-1.082033,0.063342


In [0]:
selected_categorical_cybersecurity_sample.head()

,Source IP Address,Destination IP Address
8281,52.16.186.255,143.159.62.60
28232,161.108.110.93,171.250.86.109
14078,2.28.162.6,29.219.167.39
16368,54.22.11.21,37.134.249.227
11039,135.14.191.235,71.201.67.196


In [0]:
#writing to the databricks S3 bucket
scaled_numerical_cybersecurity_sample.to_csv('/Volumes/dims_redux/default/csv_data/scaled_numerical_cybersecurity_sample.csv', index=False)

## Processing data from the NGA

In [0]:
nga_art_sample.columns

Index(['iiifurl', 'iiifthumburl', 'accessioned', 'title',
       'displayDate_created', 'roletype', 'role', 'forwarddisplayname',
       'birthyear', 'deathyear', 'ulanid', 'artistofngaobject', 'nationality',
       'constituenttype', 'beginyear_artistAssigned', 'endyear_artistAssigned',
       'country_artistAssigned', 'zipcode_artistAssigned', 'medium',
       'dimensions', 'inscription', 'markings', 'attribution',
       'visualBrowserClassification', 'parentID', 'isVirtual', 'portfolio',
       'series', 'volume', 'watermarks', 'uuid', 'viewtype', 'sequence',
       'width', 'height', 'maxpixels', 'assistivetext', 'depictstmsobjectid',
       'objectid', 'constituentid'],
      dtype='object')

In [0]:
# selecting numerical columns from the dataset
selected_art_sample = nga_art_sample.loc[:, ['accessioned', 'birthyear', 'deathyear', 'width', 'height', 'medium', 'viewtype']]

### Filling in missing values

In [0]:
# checking for null values
selected_art_sample.isna().sum()

accessioned      2
birthyear        7
deathyear      212
width            4
height           6
medium           2
viewtype         4
dtype: int64

In [0]:
selected_art_sample.reset_index(drop=False, inplace=True)

In [0]:
# imputing null values with mean and also converting the columns to int
selected_art_sample['birthyear'] = pd.to_numeric(selected_art_sample['birthyear'], errors='coerce')
selected_art_sample['birthyear'].fillna(selected_art_sample['birthyear'].mean(), inplace=True)

selected_art_sample['deathyear'] = pd.to_numeric(selected_art_sample['deathyear'], errors='coerce')
selected_art_sample['deathyear'].fillna(selected_art_sample['deathyear'].mean(), inplace=True)

selected_art_sample['width'] = pd.to_numeric(selected_art_sample['width'], errors='coerce')
selected_art_sample['width'].fillna(selected_art_sample['width'].mean(), inplace=True)

selected_art_sample['height'] = pd.to_numeric(selected_art_sample['height'], errors='coerce')
selected_art_sample['height'].fillna(selected_art_sample['height'].mean(), inplace=True)

# imputing the categorical string column 'medium' with the most frequent value
selected_art_sample['medium'].fillna(selected_art_sample['medium'].value_counts().index[0], inplace=True)
# imputing the categorical string column 'viewtype' with the most frequent value
selected_art_sample['viewtype'].fillna(selected_art_sample['viewtype'].value_counts().index[0], inplace=True)


In [0]:
selected_art_sample.loc[:, ['medium', 'viewtype']].describe()

,medium,viewtype
count,1256,1256
unique,393,3
top,gelatin silver print,primary
freq,180,1243


In [0]:
selected_art_sample.describe()

,index,birthyear,deathyear,width,height
count,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000
mean,47496.172771,1867.110488,1935.539272,4171.233227,4322.582400
std,61903.407717,109.448792,107.196537,2252.697994,2169.317929
min,0.000000,1200.000000,1299.000000,640.000000,1092.000000
25%,312.750000,1855.000000,1935.539272,3066.000000,3202.750000
50%,626.500000,1902.000000,1969.000000,3589.000000,4000.000000
75%,91973.500000,1923.000000,1999.000000,4000.000000,4000.000000
max,199783.000000,1980.000000,2021.000000,33232.000000,24027.000000


### Scaling the selected art sample for use in Dimensionality Reduction

In [0]:
# applying the standard scaler to the numerical data
scaler = StandardScaler()
selected_numerical_art_sample = selected_art_sample.loc[:, ['birthyear', 'deathyear', 'width', 'height']]
scaler.fit(selected_numerical_art_sample)
scaled_numerical_art_sample = scaler.transform(selected_numerical_art_sample)
scaled_numerical_art_sample = pd.DataFrame(scaler.transform(selected_numerical_art_sample), columns=selected_numerical_art_sample.columns)

In [0]:
scaled_numerical_art_sample.describe()

,birthyear,deathyear,width,height
count,1.256000e+03,1.256000e+03,1.256000e+03,1.256000e+03
mean,1.946072e-15,1.810300e-16,-1.258724e-16,-1.074866e-16
std,1.000398e+00,1.000398e+00,1.000398e+00,1.000398e+00
min,-6.097612e+00,-5.940423e+00,-1.568182e+00,-1.489809e+00
25%,-1.106939e-01,0.000000e+00,-4.908219e-01,-5.164197e-01
50%,3.189017e-01,3.122681e-01,-2.585634e-01,-1.487615e-01
75%,5.108487e-01,5.922393e-01,-7.604279e-02,-1.487615e-01
max,1.031848e+00,7.975516e-01,1.290557e+01,9.086850e+00


In [0]:
# writing to the databricks s3 bucket
scaled_numerical_art_sample.to_csv('/Volumes/dims_redux/default/csv_data/scaled_numerical_art_sample.csv', index=False)

# Simpler Linear Methods
## PCA (Principal Component Analysis)

In [0]:
# applying PCA to the scaled data
from sklearn.decomposition import PCA
def apply_pca(scaled_df, n_components=2):
    pca = PCA(n_components=n_components)
    pca.fit(scaled_df)
    pca_df = pca.transform(scaled_df)
    pca_df = pd.DataFrame(pca_df, columns=['PC1', 'PC2'])
    return pca_df

In [0]:
pca_cyber = apply_pca(scaled_numerical_cybersecurity_sample, n_components=2)
pca_cyber.head()

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

,PC1,PC2
0,-0.947665,-0.435792
1,0.144337,-2.327878
2,0.689013,0.931354
3,-0.263628,-1.069374
4,0.157555,0.966615


## SVD (Singular Value Decomposition)

In [0]:
from sklearn.decomposition import TruncatedSVD

def apply_svd(scaled_df, n_components=2):
    svd = TruncatedSVD(n_components=n_components)
    svd.fit(scaled_df)
    svd_df = svd.transform(scaled_df)
    svd_df = pd.DataFrame(svd_df, columns=[f'SVD{i+1}' for i in range(n_components)])
    return svd_df

In [0]:
svd_cyber = apply_svd(scaled_numerical_cybersecurity_sample)
svd_cyber.head()

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

,SVD1,SVD2
0,-0.947665,-0.435792
1,0.144337,-2.327878
2,0.689013,0.931354
3,-0.263628,-1.069374
4,0.157555,0.966615


## MDS (Multidimensional Scaling)

In [0]:
from sklearn.manifold import MDS

def apply_mds(scaled_df, n_components=2):
    mds = MDS(n_components=n_components)
    mds_df = mds.fit_transform(scaled_df)
    mds_df = pd.DataFrame(mds_df, columns=[f'MDS{i+1}' for i in range(n_components)])
    return mds_df

In [0]:
mds_art = apply_mds(scaled_numerical_art_sample)
mds_art.head()

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

,MDS1,MDS2
0,0.072967,-0.932624
1,0.072967,-0.932624
2,0.083065,-0.812559
3,0.083065,-0.812559
4,-1.114959,-0.171940


## ISOMap (Isometric Mapping)

In [0]:
from sklearn.manifold import Isomap

def apply_isomap(scaled_df, n_components=2, n_neighbors=5):
    isomap = Isomap(n_components=n_components, n_neighbors=n_neighbors)
    isomap_df = isomap.fit_transform(scaled_df)
    isomap_df = pd.DataFrame(isomap_df, columns=[f'ISOMap{i+1}' for i in range(n_components)])
    return isomap_df

In [0]:
isomap_art = apply_isomap(scaled_numerical_art_sample)
isomap_art.head()

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

,ISOMap1,ISOMap2
0,-1.478888,-1.082827
1,-1.478888,-1.082827
2,-1.039372,-0.830713
3,-1.039372,-0.830713
4,-0.633842,-0.758728


## LLE (Locally Linear Embedding)

In [0]:
from sklearn.manifold import LocallyLinearEmbedding

def apply_lle(scaled_df, n_components=2, n_neighbors=5):
    lle = LocallyLinearEmbedding(n_components=n_components, n_neighbors=n_neighbors, eigen_solver = 'dense')
    lle_df = lle.fit_transform(scaled_df)
    lle_df = pd.DataFrame(lle_df, columns=[f'LLE{i+1}' for i in range(n_components)])
    return lle_df

In [0]:
lle_art = apply_lle(scaled_numerical_art_sample)
lle_art.head()

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

,LLE1,LLE2
0,0.004696,0.012536
1,0.004696,0.012536
2,-0.011816,-0.003212
3,-0.011816,-0.003212
4,0.006529,-0.015254


# Complex Multi-linear Methods
## UMAP (Uniform Manifold Approximation and Projection)

In [0]:
%pip install umap-learn

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.9 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86837 sha256=d322549b5866f1c1f3783e00688e0a33e051ff861fe95dcd99a0eb02a0726f7d
  Stored in directory: /root/.cache/pip/wheels/e5/c1/ee/d66f7fa7b04af4405e561e6836c203b5eea7e85c4a6fa29946
Successfully built umap-learn
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from umap import UMAP

def apply_umap(scaled_df, n_components=2, n_neighbors=5):
    umap = UMAP(n_components=n_components, n_neighbors=n_neighbors)
    umap_df = umap.fit_transform(scaled_df)
    umap_df = pd.DataFrame(umap_df, columns=[f'UMAP{i+1}' for i in range(n_components)])
    return umap_df

2023-11-28 00:57:40.088530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [0]:
umap_cyber = apply_umap(selected_numerical_cybersecurity_sample)
umap_cyber.head()

,UMAP1,UMAP2
0,5.941579,-1.311825
1,15.317224,7.929322
2,0.894686,-3.161721
3,13.830988,2.049630
4,-2.782944,10.725903


In [0]:
# saving to the databricks s3 bucket
umap_cyber.to_csv('/Volumes/dims_redux/default/csv_data/umap_cybersecurity.csv', index=False)

## t-SNE (t-Distributed Stochastic Neighbor Embedding)

In [0]:
from sklearn.manifold import TSNE

def apply_tsne(scaled_df, n_components=2, perplexity=30, learning_rate=200):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, learning_rate=learning_rate)
    tsne_df = tsne.fit_transform(scaled_df)
    tsne_df = pd.DataFrame(tsne_df, columns=[f't-SNE{i+1}' for i in range(n_components)])
    return tsne_df

In [0]:
tsne_art = apply_tsne(scaled_numerical_art_sample)
tsne_art.head()

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

,t-SNE1,t-SNE2
0,6.881261,-39.364033
1,6.881261,-39.364033
2,-6.852994,-32.309406
3,-6.852994,-32.309406
4,3.986995,1.049295


In [0]:
# uploading to the databricks s3 bucket
tsne_art.to_csv('/Volumes/dims_redux/default/csv_data/tsne_art.csv', index=False)